In [ ]:
# https://ps-porn.com/sapi/hKB_d-rTkAad_kmVfWjfFA/1739591699/Activity_Event.get?_method=Activity_Event.get&tz=2&date=2019-08-31%2018%3A51%3A26&direction=prev&types[1]=1&types[2]=1&types[3]=0&types[4]=0&types[5]=1&types[6]=1&types[7]=1&limit=18&returnNewFormat=true&transitParameters[v1]=ykYa8ALmUD
# https://ps-porn.com/sapi/hKB_d-rTkAad_kmVfWjfFA/1739591699/Activity_Event.get?_method=Activity_Event.get&tz=2&date=2019-10-16%2020%3A32%3A07&direction=prev&types[1]=1&types[2]=1&types[3]=0&types[4]=0&types[5]=1&types[6]=1&types[7]=1&limit=18&returnNewFormat=true&transitParameters[v1]=ykYa8ALmUD

In [5]:
import os
import json
import polars as pl
from pathlib import Path

def read_json_files(directory):
    """Read all JSON files from a directory and return a list of their contents"""
    json_data = []
    directory_path = Path(directory)
    
    # Iterate through all files in directory
    for file_path in directory_path.glob('*.json'):
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
                json_data.append(data)
        except Exception as e:
            print(f"Error reading {file_path}: {str(e)}")
    
    return json_data

# Read all JSON files
directory = "H:\\Parquet Data\\PS-Porn"
json_files = read_json_files(directory)

In [ ]:
events = []
for json_file in json_files:
    current_events = json_file["response"]["collection"]
    events.extend(current_events)

In [ ]:
events_df = pl.DataFrame(events)
events_df = events_df.with_columns(
    pl.col("_typedParams").struct.field("id").alias("id"),
    pl.col("_typedParams").struct.field("thumb").alias("thumb"),
    pl.col("startDate").str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S").dt.date().alias("date")
)
events_df = events_df.select(["id", "title", "date", "startDate"])
events_df

In [38]:
events_df.write_parquet("H:\\Parquet Data\\PS-Porn\\events_df.parquet")

In [ ]:
events_df.filter(pl.col("id").is_in([7712298, 7735906, 7816503, 7672222]))